# Lab 3.2 - Agentic AI with CrewAI + watsonx.ai

This notebook demonstrates how to build a multi-agent system using **CrewAI's native watsonx.ai integration**.

## What You'll Learn

- How to use CrewAI's native `LLM` class with watsonx.ai (no wrappers needed!)
- Creating custom tools for agents (RAG service and calculator)
- Building a collaborative agent system
- Using agents to solve complex tasks

## Architecture

1. **watsonx.ai** - IBM's foundation model platform (Granite models)
2. **CrewAI** - Multi-agent orchestration framework with native watsonx.ai support
3. **Custom Tools** - RAG service and calculator tools
4. **Accelerator RAG API** - Backend RAG service for knowledge retrieval

## Key Improvement

✨ **This notebook uses CrewAI's native `LLM` class** - no LangChain wrappers needed!

```python
# Old way (LangChain wrapper):
from langchain_ibm import WatsonxLLM
llm = WatsonxLLM(model_id=..., url=..., apikey=..., ...)

# New way (CrewAI native):
from crewai import LLM
llm = LLM(model="watsonx/ibm/granite-3-8b-instruct", api_key=..., base_url=..., ...)
```

---

## 1. Setup and Installation

### Google Colab Compatibility

This notebook is designed to work both locally and in Google Colab.

In [ ]:
# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("✓ Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("✓ Running in local environment")

In [ ]:
# Install required packages
# Note: No need for langchain-ibm or ibm-watsonx-ai!
!pip install -q crewai crewai-tools requests python-dotenv

print("✅ Packages installed!")
print("\nInstalled:")
print("  - CrewAI (with native watsonx.ai support)")
print("  - CrewAI Tools")
print("  - Requests (for RAG API)")
print("  - Python-dotenv (optional)")

## 2. Configure watsonx.ai Credentials

To use watsonx.ai, you need:

1. **API Key** - Get it from [IBM Cloud](https://cloud.ibm.com/iam/apikeys)
2. **Project ID** - From your watsonx.ai project
3. **URL** - Regional endpoint (default: us-south)

### How to Get Your Credentials

1. Go to [IBM Cloud](https://cloud.ibm.com)
2. Navigate to watsonx.ai
3. Create or open a project
4. Copy your Project ID from the project settings
5. Create an API key from IBM Cloud IAM

### Security Best Practices

- ✅ Use environment variables for credentials
- ✅ Never commit credentials to version control
- ✅ Use `.env` files locally (add to `.gitignore`)
- ✅ Rotate API keys regularly

In [ ]:
import os
from getpass import getpass

# Disable CrewAI telemetry for cleaner output
os.environ["CREWAI_DISABLE_TELEMETRY"] = "true"
os.environ["CREWAI_TELEMETRY"] = "false"

# Configuration for watsonx.ai
WATSONX_URL = os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com")

if not os.getenv("WATSONX_API_KEY"):
    WATSONX_API_KEY = getpass("Enter your watsonx.ai API Key: ")
else:
    WATSONX_API_KEY = os.getenv("WATSONX_API_KEY")

if not os.getenv("WATSONX_PROJECT_ID"):
    WATSONX_PROJECT_ID = getpass("Enter your watsonx.ai Project ID: ")
else:
    WATSONX_PROJECT_ID = os.getenv("WATSONX_PROJECT_ID")

# Store in environment for easy access
os.environ["WATSONX_API_KEY"] = WATSONX_API_KEY
os.environ["WATSONX_URL"] = WATSONX_URL
os.environ["WATSONX_PROJECT_ID"] = WATSONX_PROJECT_ID

# Model configuration - Native CrewAI format!
WATSONX_MODEL = "watsonx/ibm/granite-3-8b-instruct"

# Accelerator API URL (set this to your RAG service endpoint)
ACCELERATOR_API_URL = os.getenv("ACCELERATOR_API_URL", "http://localhost:8000/ask")

print("✓ Configuration loaded")
print(f"  Model: {WATSONX_MODEL}")
print(f"  URL: {WATSONX_URL}")
print(f"  RAG API: {ACCELERATOR_API_URL}")
print("\n✨ Using CrewAI's native watsonx.ai integration!")

## 3. Initialize watsonx.ai LLM with CrewAI

### The Modern Way: Native Integration

CrewAI now has **native support** for IBM watsonx.ai. No LangChain wrappers needed!

#### Advantages:
- ✅ **Simpler code** - No custom wrappers
- ✅ **Fewer dependencies** - No langchain-ibm needed
- ✅ **Better maintained** - Official CrewAI support
- ✅ **More flexible** - Easy to customize per agent

In [ ]:
from crewai import LLM

# Create watsonx.ai LLM using CrewAI's native class
watsonx_llm = LLM(
    model=WATSONX_MODEL,                      # Format: watsonx/provider/model
    api_key=os.environ["WATSONX_API_KEY"],
    base_url=os.environ["WATSONX_URL"],
    project_id=os.environ["WATSONX_PROJECT_ID"],
    temperature=0.3,                           # Balanced creativity/focus
    max_tokens=1000,                          # Maximum response length
)

print("✅ watsonx.ai LLM initialized successfully!")
print("\n📊 Configuration:")
print(f"  Model: {WATSONX_MODEL}")
print(f"  Temperature: 0.3 (balanced)")
print(f"  Max Tokens: 1000")
print("  Integration: CrewAI Native")
print("\n🎉 Ready to power our agents!")

## 4. Define Custom Tools

We'll create two tools for our agents:

1. **RAG Service Tool** - Queries the accelerator RAG API for knowledge retrieval
2. **Calculator Tool** - Performs safe arithmetic calculations

These tools give our agents the ability to access external knowledge and perform computations.

### Tool Design Best Practices

- Clear, descriptive docstrings (agents read these!)
- Proper error handling
- Input validation
- Meaningful return messages

In [ ]:
import json
import ast
import operator as op
from typing import Any
import requests
from crewai_tools import tool

@tool("rag_service_tool")
def rag_service_tool(question: str) -> str:
    """
    Query the RAG (Retrieval-Augmented Generation) service to answer questions
    based on enterprise knowledge base.
    
    Use this tool when you need to:
    - Answer questions about specific documents or knowledge
    - Retrieve factual information from the knowledge base
    - Get context-aware responses based on enterprise data
    - Find information about watsonx.ai, RAG systems, or enterprise AI
    
    Args:
        question: The question to ask the RAG service (be specific!)
    
    Returns:
        Formatted string containing the answer and citations from the knowledge base
    """
    try:
        payload = {"question": question}
        resp = requests.post(ACCELERATOR_API_URL, json=payload, timeout=60)
        resp.raise_for_status()
        data = resp.json()
        
        answer = data.get("answer") or data.get("result") or "No answer available"
        citations = data.get("citations") or data.get("chunks") or []
        
        result = f"ANSWER: {answer}\n"
        if citations:
            result += f"\nCITATIONS: {json.dumps(citations, indent=2)}"
        
        return result
    except requests.exceptions.RequestException as e:
        return f"Error connecting to RAG service: {str(e)}. Make sure ACCELERATOR_API_URL is configured correctly."
    except Exception as e:
        return f"Error calling RAG service: {str(e)}"


# Safe calculator implementation using AST
_allowed_operators = {
    ast.Add: op.add,
    ast.Sub: op.sub,
    ast.Mult: op.mul,
    ast.Div: op.truediv,
    ast.Pow: op.pow,
}


def _eval_ast(node):
    """Safely evaluate an AST node."""
    if isinstance(node, ast.Num):  # Python 3.7 compatibility
        return node.n
    if isinstance(node, ast.Constant):  # Python 3.8+
        return node.value
    if isinstance(node, ast.BinOp) and type(node.op) in _allowed_operators:
        return _allowed_operators[type(node.op)](_eval_ast(node.left), _eval_ast(node.right))
    if isinstance(node, ast.UnaryOp) and isinstance(node.op, (ast.UAdd, ast.USub)):
        value = _eval_ast(node.operand)
        return +value if isinstance(node.op, ast.UAdd) else -value
    raise ValueError("Unsupported expression")


@tool("calculator_tool")
def calculator_tool(expression: str) -> str:
    """
    Safely evaluate arithmetic expressions.
    
    Use this tool when you need to:
    - Perform mathematical calculations
    - Evaluate arithmetic expressions
    - Do numerical computations
    - Calculate totals, averages, or other numeric operations
    
    Supported operations: +, -, *, /, ** (power)
    
    Examples:
    - Simple: '2 + 2', '10 - 5'
    - Complex: '(15 + 25) * 3', '2 ** 8'
    - Division: '100 / 4', '17 / 3'
    
    Args:
        expression: Mathematical expression to evaluate (string)
    
    Returns:
        Result of the calculation or error message
    """
    try:
        # Remove any whitespace and validate
        expression = expression.strip()
        if not expression:
            return "Error: Empty expression provided"
        
        parsed = ast.parse(expression, mode="eval")
        result = _eval_ast(parsed.body)
        return f"Result: {result}"
    except SyntaxError:
        return f"Error: Invalid mathematical expression '{expression}'"
    except ZeroDivisionError:
        return "Error: Division by zero"
    except ValueError as e:
        return f"Error: {str(e)}"
    except Exception as e:
        return f"Error evaluating expression: {str(e)}"


print("✅ Custom tools defined successfully!")
print("\n🔧 Available Tools:")
print("  1. RAG Service Tool - Knowledge retrieval")
print("  2. Calculator Tool - Mathematical operations")

## 5. Create CrewAI Agents with Native watsonx.ai LLM

Now we'll create specialized agents, each powered by watsonx.ai through CrewAI's native integration:

1. **Research Agent** - Uses RAG service to find information
2. **Calculator Agent** - Handles mathematical computations
3. **Support Agent** - Orchestrates and delegates to specialists

Each agent has a specific role, goal, and backstory that guides its behavior.

### Agent Design Principles

- **Clear Role**: Defines what the agent does
- **Specific Goal**: What the agent aims to achieve
- **Rich Backstory**: Context that shapes decision-making
- **Right Tools**: Only tools needed for the role
- **Strategic Delegation**: When to hand off to specialists

In [ ]:
from crewai import Agent, Task, Crew, Process

# Research Agent - specializes in information retrieval
research_agent = Agent(
    role="Research Specialist",
    goal="Find accurate and relevant information from the knowledge base to answer user questions",
    backstory="""
        You are an expert research specialist with deep knowledge of watsonx.ai and 
        enterprise AI systems. You excel at finding relevant information and providing 
        well-cited, accurate answers. You always use the RAG service to ground your 
        responses in factual data from the knowledge base.
    """,
    tools=[rag_service_tool],
    llm=watsonx_llm,  # Powered by watsonx.ai natively!
    verbose=True,
    allow_delegation=False
)

# Calculator Agent - specializes in mathematical operations
calculator_agent = Agent(
    role="Mathematics Specialist",
    goal="Perform accurate mathematical calculations and explain the results clearly",
    backstory="""
        You are a mathematics expert who excels at performing calculations and 
        explaining mathematical concepts. You use the calculator tool for all 
        numerical computations to ensure 100% accuracy. You break down complex
        calculations into steps when helpful.
    """,
    tools=[calculator_tool],
    llm=watsonx_llm,  # Same LLM, different specialization!
    verbose=True,
    allow_delegation=False
)

# Support Agent - orchestrates and delegates to specialists
support_agent = Agent(
    role="Workshop Support Coordinator",
    goal="Help users by using the right tools or delegating to specialist agents as needed",
    backstory="""
        You are a helpful coordinator who understands when to use the RAG service 
        for knowledge questions and when to use the calculator for math problems. 
        You have access to both tools and can intelligently decide which is most 
        appropriate for each question. You're also skilled at combining results 
        from multiple tools when needed.
    """,
    tools=[rag_service_tool, calculator_tool],
    llm=watsonx_llm,  # All agents use the same watsonx.ai model!
    verbose=True,
    allow_delegation=True  # Can delegate to specialists
)

print("✅ Agents created successfully!")
print("\n🤖 Agent Team:")
print("  1. Research Specialist (RAG tool)")
print("  2. Mathematics Specialist (Calculator tool)")
print("  3. Support Coordinator (Both tools + delegation)")
print("\n✨ All powered by IBM watsonx.ai via CrewAI's native integration!")

## 6. Define Tasks and Create Crew

We'll create a flexible task factory that can be used for any user question.
The support agent will automatically choose the right tool or delegate to specialists.

In [ ]:
def create_support_task(question: str) -> Task:
    """
    Create a task for the support agent to handle a user question.
    
    The agent will automatically:
    - Use RAG service for knowledge questions
    - Use calculator for mathematical questions
    - Combine tools when needed
    - Delegate to specialists if appropriate
    
    Args:
        question: The user's question
    
    Returns:
        Task object configured for the support agent
    """
    return Task(
        description=f"""
            Answer the following user question: {question}
            
            Guidelines:
            - For factual or knowledge-based questions, use the RAG service tool
            - For mathematical calculations, use the calculator tool
            - Provide clear, concise answers with proper citations when applicable
            - If you use a tool, briefly explain what you found or calculated
            - Be helpful and professional in your response
        """,
        expected_output="A clear, helpful answer with explanation of approach used",
        agent=support_agent,
    )


# Create the crew
crew = Crew(
    agents=[support_agent, research_agent, calculator_agent],
    tasks=[],  # Tasks will be added dynamically
    process=Process.sequential,
    verbose=True,
    memory=False,  # Disable memory for simpler demo
)

print("✅ Crew initialized successfully!")
print("\n📋 Crew Configuration:")
print("  Process: Sequential")
print("  Agents: 3 (Support, Research, Calculator)")
print("  Memory: Disabled")
print("  Tasks: Added dynamically per question")

## 7. Test the Multi-Agent System

Let's test our crew with different types of questions to see how it handles them.

The support agent will automatically:
- Choose the right tool
- Delegate to specialists when needed
- Combine information from multiple sources

### Test 1: Knowledge Question (Should use RAG service)

This question requires information retrieval from the knowledge base.

In [ ]:
question_1 = "What is Retrieval-Augmented Generation (RAG) and why is it important?"

print("=" * 80)
print(f"USER QUESTION: {question_1}")
print("=" * 80)
print("\nExpected behavior: Agent should use RAG service tool\n")

task_1 = create_support_task(question_1)
crew.tasks = [task_1]
result_1 = crew.kickoff()

print("\n" + "=" * 80)
print("FINAL ANSWER:")
print("=" * 80)
print(result_1)

### Test 2: Mathematical Question (Should use calculator)

This question requires numerical computation.

In [ ]:
question_2 = "Calculate: (15 + 25) * 3 - 10"

print("=" * 80)
print(f"USER QUESTION: {question_2}")
print("=" * 80)
print("\nExpected behavior: Agent should use calculator tool\n")

task_2 = create_support_task(question_2)
crew.tasks = [task_2]
result_2 = crew.kickoff()

print("\n" + "=" * 80)
print("FINAL ANSWER:")
print("=" * 80)
print(result_2)

### Test 3: Enterprise AI Question

A more complex question about enterprise AI systems.

In [ ]:
question_3 = "What are the benefits of using watsonx.ai for enterprise AI applications?"

print("=" * 80)
print(f"USER QUESTION: {question_3}")
print("=" * 80)
print("\nExpected behavior: Agent should use RAG service for enterprise knowledge\n")

task_3 = create_support_task(question_3)
crew.tasks = [task_3]
result_3 = crew.kickoff()

print("\n" + "=" * 80)
print("FINAL ANSWER:")
print("=" * 80)
print(result_3)

### Test 4: Complex Calculation

A more complex mathematical expression.

In [ ]:
question_4 = "What is 2 to the power of 10, divided by 4?"

print("=" * 80)
print(f"USER QUESTION: {question_4}")
print("=" * 80)
print("\nExpected behavior: Agent should use calculator (2**10 / 4)\n")

task_4 = create_support_task(question_4)
crew.tasks = [task_4]
result_4 = crew.kickoff()

print("\n" + "=" * 80)
print("FINAL ANSWER:")
print("=" * 80)
print(result_4)

## 8. Interactive Question Answering

Now you can ask your own questions!

The crew will automatically:
- Determine the question type
- Use the appropriate tool
- Provide well-formatted answers

In [ ]:
def ask_crew(question: str):
    """
    Ask a question to the crew and get an answer.
    
    Args:
        question: Your question (can be knowledge-based or mathematical)
    
    Returns:
        The crew's answer
    """
    print("=" * 80)
    print(f"QUESTION: {question}")
    print("=" * 80)
    print()
    
    task = create_support_task(question)
    crew.tasks = [task]
    result = crew.kickoff()
    
    print("\n" + "=" * 80)
    print("ANSWER:")
    print("=" * 80)
    print(result)
    
    return result


# Try it out!
# Examples:
# ask_crew("What is multi-agent AI?")
# ask_crew("Calculate 456 * 789")
# ask_crew("How does watsonx.ai support RAG workflows?")

print("✅ Interactive function ready!")
print("\n💡 Try asking questions:")
print("  - Knowledge: ask_crew('What is multi-agent AI?')")
print("  - Math: ask_crew('Calculate 123 * 456')")
print("  - Complex: ask_crew('Explain watsonx.ai RAG capabilities')")

## 9. Summary and Key Takeaways

### What We Learned

1. **Native Integration**: CrewAI's `LLM` class provides seamless watsonx.ai integration
2. **No Wrappers Needed**: Simpler, cleaner code without LangChain dependencies
3. **Custom Tools**: Extended agent capabilities with RAG and calculator tools
4. **Multi-Agent Architecture**: Built specialized agents with distinct roles
5. **Agent Collaboration**: Agents work together and delegate based on expertise

### Modern vs Legacy Approach

```python
# ❌ Old Way (LangChain wrapper)
from langchain_ibm import WatsonxLLM
llm = WatsonxLLM(
    model_id="ibm/granite-3-8b-instruct",
    url=WATSONX_URL,
    apikey=WATSONX_APIKEY,
    project_id=WATSONX_PROJECT_ID,
    params={...}
)

# ✅ New Way (CrewAI native)
from crewai import LLM
llm = LLM(
    model="watsonx/ibm/granite-3-8b-instruct",
    api_key=WATSONX_API_KEY,
    base_url=WATSONX_URL,
    project_id=WATSONX_PROJECT_ID,
    temperature=0.3
)
```

### Key Components

- **watsonx.ai**: IBM's foundation model platform (Granite models)
- **CrewAI**: Multi-agent orchestration with native watsonx.ai support
- **Custom Tools**: Extensions that give agents special capabilities
- **RAG Service**: External knowledge retrieval system

### Best Practices

1. **Clear Agent Roles**: Define specific roles and responsibilities
2. **Appropriate Tools**: Give agents only the tools they need
3. **Rich Backstories**: Provide context that shapes decision-making
4. **Strategic Delegation**: Use delegation for complex workflows
5. **Error Handling**: Include proper error handling in tools
6. **Good Documentation**: Write clear docstrings for tools

### Advantages of Native Integration

✅ **Fewer dependencies** - No langchain-ibm or ibm-watsonx-ai needed  
✅ **Simpler code** - Direct integration with CrewAI  
✅ **Better maintained** - Official CrewAI support  
✅ **More flexible** - Easy to customize per agent  
✅ **Production ready** - Built for enterprise use  

### Next Steps

- Try the LangGraph notebook for a different agent architecture
- Experiment with the Langflow visual builder
- Create your own custom tools and agents
- Build more complex multi-agent workflows
- Explore different watsonx.ai models (Granite, Llama, Mixtral)

---

**Course**: Multi-Agent Systems with watsonx.ai  
**Lab**: 3.2 - CrewAI Integration (Refactored)  
**Platform**: Compatible with Google Colab and local environments  
**Integration**: Native CrewAI LLM class (no wrappers!) ✨

## Appendix: Quick Reference

### Native watsonx.ai Model Format

```python
# Format: watsonx/provider/model-name

# IBM Granite Models
"watsonx/ibm/granite-3-8b-instruct"      # Efficient, balanced
"watsonx/ibm/granite-13b-instruct-v2"    # Larger, more capable
"watsonx/ibm/granite-3-2b-instruct"      # Smaller, faster

# Meta Llama Models
"watsonx/meta-llama/llama-3-70b-instruct"  # Very capable
"watsonx/meta-llama/llama-3-8b-instruct"   # Efficient

# Mistral Models
"watsonx/mistralai/mixtral-8x7b-instruct"  # Mixture of experts
```

### Creating an LLM Instance

```python
from crewai import LLM

llm = LLM(
    model="watsonx/ibm/granite-3-8b-instruct",
    api_key=os.environ["WATSONX_API_KEY"],
    base_url=os.environ["WATSONX_URL"],
    project_id=os.environ["WATSONX_PROJECT_ID"],
    temperature=0.3,
    max_tokens=1000,
)
```

### Creating an Agent

```python
from crewai import Agent

agent = Agent(
    role="Your Role",
    goal="Your Goal",
    backstory="Your Backstory",
    tools=[your_tools],
    llm=llm,
    verbose=True,
)
```

### Creating a Tool

```python
from crewai_tools import tool

@tool("tool_name")
def your_tool(param: str) -> str:
    """Clear description for the agent."""
    # Your tool logic
    return result
```

### Environment Variables

```bash
WATSONX_API_KEY=your_api_key
WATSONX_URL=https://us-south.ml.cloud.ibm.com
WATSONX_PROJECT_ID=your_project_id
ACCELERATOR_API_URL=http://localhost:8000/ask
```